In [1]:
# import modules
import copy
import time
from sys import path
from collections import defaultdict

In [2]:
import pandas as pd
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
path.append('../code/src')

In [4]:
from post_rec import PostRec
from inter_rec import InterRec
from TF_algo import SVDtf
from evaluate import Evaluation as ev

In [5]:
# constants
# file path
input_path = "../data/"
output_path = '../result/'
rate_file = 'reduced_rating_data.csv'
attr_file = 'recipe_data.csv'

# user/item max
user_max = 1000
food_max = 5000

# const count
const_count = 3

In [6]:
# Dictionary keys
rs1 = 'w/o Constraint'
rs2 = 'Post-Single-'
rs3 = 'Inter-Single-'
val_r1 = 'RMSE'
val_r2 = 'NDCG'
val_t1 = 'train time(s)'
val_t2 = 'exec time(s)'

In [7]:
val_dict = {
    val_r1: 0,
    val_r2: 0,
    val_t1: 0,
    val_t2: 0
}

keys = [rs1]
keys = keys + [rs2 + str(i) for i in range (1, 4)]
keys = keys + [rs3 + str(i) for i in range (1, 4)]

result = {key: copy.deepcopy(val_dict) for key in keys}

In [8]:
# read previous result
result_df = pd.read_csv(output_path + 'eval_RMSE.csv', index_col = 0).transpose()
result.update(result_df.to_dict())
result

{'w/o Constraint': {'RMSE': 0.8036993430910471,
  'train time(s)': 0.2642946243286133,
  'exec time(s)': 14.285892248153688},
 'Post-Single-1': {'RMSE': 4.699165956844706,
  'train time(s)': 0.262630542119344,
  'exec time(s)': 48.98440051078797},
 'Post-Single-2': {'RMSE': 4.699165956844706,
  'train time(s)': 0.2546525796254476,
  'exec time(s)': 16.857391834259033},
 'Post-Single-3': {'RMSE': 0.8389188603754039,
  'train time(s)': 0.256984790166219,
  'exec time(s)': 31.852389176686604},
 'Inter-Single-1': {'RMSE': 3.9437595397206646,
  'train time(s)': 254.9055339495341,
  'exec time(s)': 9.826252222061155},
 'Inter-Single-2': {'RMSE': 2.917776429023384,
  'train time(s)': 273.18191345532733,
  'exec time(s)': 18.86075750986735},
 'Inter-Single-3': {'RMSE': 0.8273979310684251,
  'train time(s)': 339.7311069170634,
  'exec time(s)': 20.7761754989624}}

# Evaluate RS w/o Constraint
Check Suprise SVD performance with given rating data

In [9]:
# Source From: https://github.com/NicolasHug/Surprise/blob/master/examples/top_n_recommendations.py
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    ret = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        ret[uid] = [i for i, r in top_n[uid]]
    
    return ret

In [10]:
def run_SVD():    
    # get data
    reader = Reader(line_format='user item rating', sep=',', rating_scale=(0, 5))
    data = Dataset.load_from_file(input_path + rate_file, reader = reader)
    
    train_set, test_set = train_test_split(data, test_size=0.25)
    anti_set = train_set.build_anti_testset()
    
    # use SVD algorithm
    algo = SVD()
    
    # train
    start = time.time()
    algo.fit(train_set)
    t1 = time.time() - start
    
    # predict rating for test-set
    predict_test = algo.test(test_set)
    r1 = ev.calculate_rmse(predict_test)

    # get top-n for anti-test-set
    start = time.time()
    predict_full = algo.test(anti_set)
    top_n = get_top_n(predict_full)
    t2 = time.time() - start
    
    # predict nDCG for test-set
    rel_dict = defaultdict(list)
    for (u, i, r) in test_set:
        if r >= 4:
            rel_dict[int(u)].append(int(i))
    
    top_n_df = pd.DataFrame.from_dict(top_n, orient='index')
    top_n_df = top_n_df.reindex(columns=[x for x in range(0, 10)])
    r2 = ev.calculate_ndcg(rel_dict, top_n_df, 10)
    
    return r1, r2, t1, t2

In [11]:
r1, r2, t1, t2 = run_SVD()
result[rs1][val_r1] = r1
result[rs1][val_r2] = r2
result[rs1][val_t1] = t1
result[rs1][val_t2] = t2

ZeroDivisionError: division by zero

# Evaluate Post-Rec w/ Single Type Constraint

In [12]:
def run_post(ctype, idx, save_result = True):    
    # PostRec applies constraint after the rating of each item is predicted
    rec = post_rec.PostRec(input_path + rate_file, input_path + attr_file, 
                           input_path + 'const_' + str(ctype) + '.' + str(idx) + '.csv',
                            split = True)
    
    rec.get_data()  # get rating, attribute, recipe data
    
    # train with data
    start = time.time()
    rec.train()  
    t1 = time.time() - start
    
    # predict rating for test-set
    predict_test = rec.test_rmse()
    r1 = ev.calculate_rmse(predict_test)
    
    
    # get top-n for anti-test-set
    start = time.time()
    rec.test()
    top_n_df = rec.get_top_n()
    t2 = time.time() - start

    if save_result:       
        top_n_df.to_csv(output_path + 'PostRec_' + str(ctype) + '.' + str(idx) + '.csv')
        
    return r, t1, t2

In [13]:
for i in range (1, 4):
    r_sum = 0
    t1_sum = 0
    t2_sum = 0
    for j in range (1, const_count + 1):
        r, t1, t2 = run_post(i, j, True)
        r_sum = r_sum + r
        t1_sum = t1_sum + t1
        t2_sum = t2_sum + t2
        print('Const_'+str(i)+'.'+str(j)+" done")
        
    result[rs2+str(i)][val_r] = r_sum/const_count
    result[rs2+str(i)][val_t1] = t1_sum/const_count
    result[rs2+str(i)][val_t2] = t2_sum/const_count

Const_1.1 done
Const_1.2 done
Const_1.3 done
Const_2.1 done
Const_2.2 done
Const_2.3 done
Const_3.1 done
Const_3.2 done
Const_3.3 done


### Sample Dataframe

In [14]:
pd.read_csv(output_path + 'PostRec_1.1.csv', index_col = 0).head()

,0,1,2,3,4,5,6,7,8,9,i1,i2,nl
309,2773,176,4336,3047,3420,925,321,3837,2246,3659,6426,NaN,NaN
564,3003,3086,4135,1815,1262,3018,3768,4954,1174,3732,2320,NaN,NaN
560,3781,2507,4560,2789,4101,3650,1080,1493,1224,2421,7233,NaN,NaN
25,4001,3102,453,3337,2436,1734,2736,983,2355,4941,5006,NaN,NaN
873,361,4482,959,3390,4686,1791,127,1818,58,4128,6276,NaN,NaN


In [15]:
pd.read_csv(output_path + 'PostRec_2.1.csv', index_col = 0).head()

,0,1,2,3,4,5,6,7,8,9,i1,i2,nl
309,3459,4945,3428,112,229,3407,3282,4955,2210,3640,NaN,6270,NaN
564,2280,3073,1690,1951,648,240,3459,1611,4778,4018,NaN,332,NaN
560,193,1684,323,1129,3823,3910,4071,397,1734,269,NaN,5180,NaN
25,4086,3018,2301,416,2637,1023,567,1485,2549,125,NaN,332,NaN
873,1517,4017,25,309,2747,3268,3109,3510,741,1740,NaN,6270,NaN


In [16]:
pd.read_csv(output_path + 'PostRec_3.1.csv', index_col = 0).head()

,0,1,2,3,4,5,6,7,8,9,i1,i2,nl
309,1564,874,1683,1709,1740,4337,3728,2937,49,2740,NaN,NaN,"[245.1, 7.0, 7.0, 28.0, 83.0, 6.0, 1.0]"
564,4556,4778,2038,3322,4963,3047,866,4874,2499,1158,NaN,NaN,"[35.9, 1.0, 5.0, 6.0, 5.0, 3.0, 1.0]"
560,990,2377,10,2210,1984,1411,4396,4152,4450,2251,NaN,NaN,"[9.2, 0.0, 2.0, 1.0, 2.0, 0.0, 0.0]"
25,145,2968,3462,240,1287,1174,4562,2548,1430,1236,NaN,NaN,"[32.2, 5.0, 0.0, 0.0, 0.0, 2.0, 0.0]"
873,315,649,2274,2468,2319,740,2549,4572,2311,4670,NaN,NaN,"[61.8, 0.0, 42.0, 7.0, 0.0, 0.0, 5.0]"


# RMSE of Inter-Rec w/ Single Type Constraint

In [17]:
def run_inter(ctype, idx, save_result = True):    
    # PostRec applies constraint after the rating of each item is predicted
    rec = inter_rec.InterRec(input_path + rate_file, input_path + attr_file, 
                           input_path + 'const_' + str(ctype) + '.' + str(idx) + '.csv', 
                            TF_algo.SVDtf(), split = True)
    
    rec.get_data()  # get rating, attribute, recipe data
    
    # train with data
    start = time.time()
    rec.train()  
    t1 = time.time() - start
    
    # predict rating for test-set
    predict_test = rec.test_rmse()
    r = accuracy.rmse(predict_test, False)
    
    # get top-n for anti-test-set
    start = time.time()
    rec.test()
    top_n_df = rec.get_top_n()
    t2 = time.time() - start

    if save_result:       
        top_n_df.to_csv(output_path + 'InterRec_' + str(ctype) + '.' + str(idx) + '.csv')
        
    return r, t1, t2

In [18]:
for i in range (1, 4):
    r_sum = 0
    t1_sum = 0
    t2_sum = 0
    for j in range (1, const_count + 1):
        r, t1, t2 = run_inter(i, j, True)
        r_sum = r_sum + r
        t1_sum = t1_sum + t1
        t2_sum = t2_sum + t2
        print('Const_'+str(i)+'.'+str(j)+" done")
        
    result[rs3+str(i)][val_r] = r_sum/const_count
    result[rs3+str(i)][val_t1] = t1_sum/const_count
    result[rs3+str(i)][val_t2] = t2_sum/const_count

Const_1.1 done
Const_1.2 done
Const_1.3 done
Const_2.1 done
Const_2.2 done
Const_2.3 done
Const_3.1 done
Const_3.2 done
Const_3.3 done


In [19]:
pd.read_csv(output_path + 'InterRec_1.1.csv', index_col = 0).head()

,0,1,2,3,4,5,6,7,8,9,i1,i2,hl,nl
0,3510,4372,200,4301,2101,4325,855,3001,4661,4778,1124,NaN,NaN,NaN
1,82,3271,3483,2105,4535,4683,505,1918,2319,4791,6654,NaN,NaN,NaN
2,554,331,488,3845,241,1994,1357,2676,320,532,2509,NaN,NaN,NaN
3,4634,1519,12,4533,1725,700,2800,4606,1030,4581,1252,NaN,NaN,NaN
4,239,3391,1482,1202,181,925,3900,421,321,3420,6426,NaN,NaN,NaN


In [20]:
pd.read_csv(output_path + 'InterRec_2.1.csv', index_col = 0).head()

,0,1,2,3,4,5,6,7,8,9,i1,i2,hl,nl
0,228,2320,1522,296,4493,2848,4836,2203,3738,3092,NaN,840,NaN,NaN
1,647,4814,2848,125,2280,361,4573,945,4757,1715,NaN,2499,NaN,NaN
2,297,4961,3271,2949,1913,4954,361,4096,4382,1458,NaN,6270,NaN,NaN
3,4446,866,297,786,2549,1357,4894,193,3182,3732,NaN,7449,NaN,NaN
4,239,4895,2949,4847,4783,4665,3083,2274,1246,2616,NaN,5010,NaN,NaN


In [21]:
pd.read_csv(output_path + 'InterRec_3.1.csv', index_col = 0).head()

FileNotFoundError: [Errno 2] No such file or directory: '../result/InterRec_3.1.csv'

# Result

In [ ]:
result_df = pd.DataFrame.from_dict(result, orient='index', columns=['RMSE', 'train time(s)', 'exec time(s)'])
result_df.to_csv(output_path + 'eval_RMSE.csv')
result_df